# No Finetuning

In [1]:
import os
import torch
# from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from langchain import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
import pandas as pd
import numpy as np
# from peft import LoraConfig, PeftModel

In [2]:
base_model = "LDCC/LDCC-SOLAR-10.7B"
#base_model = "beomi/KoAlpaca-Polyglot-5.8B"

In [3]:
# 4-bit quantization configuration
compute_dtype = getattr(torch, "float16")
use_4bit=True

quant_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [4]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [5]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [6]:
pipe = pipeline(task="text-generation",
                model=base_model,
                tokenizer=tokenizer,
                max_length=1000,
                device_map="auto"
               )

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at LDCC/LDCC-SOLAR-10.7B and are newly initialized: ['model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.47.self_attn.rotary_emb.inv_freq', 'model.layers.42.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.35.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.29.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.33.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.44.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.38.self_attn.rotary_emb.i

In [7]:
prompt = "강마루 바닥재의 장점은 무엇인가요?"
result = pipe (f"### Question: {prompt} \n ### Answer: ")

In [8]:
result

[{'generated_text': '### Question: 강마루 바닥재의 장점은 무엇인가요? \n ### Answer: \n강마루는 나무 로 만들어 져 있어   자연스럽고 고급스러운  느낌 을 줄 수 있는  친환경적인  바닥 재입니다. 또한  내구성 이 좋고 청소 가 쉬우며  유지보수 가 비교 적 간단합니다. 그러나  강마루는 습기에  민감할 수 있으므로  설치하기  전에  바닥 의 습도 수준 을 확인하는  것이  중요합니다.'}]

In [9]:
result[0]["generated_text"].split("### Answer: ")

['### Question: 강마루 바닥재의 장점은 무엇인가요? \n ',
 '\n강마루는 나무 로 만들어 져 있어   자연스럽고 고급스러운  느낌 을 줄 수 있는  친환경적인  바닥 재입니다. 또한  내구성 이 좋고 청소 가 쉬우며  유지보수 가 비교 적 간단합니다. 그러나  강마루는 습기에  민감할 수 있으므로  설치하기  전에  바닥 의 습도 수준 을 확인하는  것이  중요합니다.']

In [10]:
# 평가 데이터
eval = []
while True:
    input_ = input()
    if input_ == "n":
        break
    else:
        eval.append(input_)

 어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?
 고층 건물을 건축할 때 철골구조가 주로 선택되는 이유는 무엇인가요?
 어떤 환경 요인이 몰딩 수정을 유발할 수 있는가요? 그리고 반점이 생긴지 1년 이내인 하자에 대해 어떤 보수작업을 해야 하나요?
 타일 바닥에서 파손된 타일을 교체하는 과정에 어떤 단계가 포함되나요? 또한, 겨울에 도배를 할 때 특별히 신경써야 할 사항이 있을까요?
 평지붕의 단점은 무엇인가요? 그리고 실크벽지의 교체 주기는 얼마나인가요?
 강마루 바닥재의 장점은 무엇인가요?
 n


In [11]:
from tqdm import tqdm
answers_nofinetuning = []
for text in tqdm(eval):
    result = pipe (f"### Question: {text} \n ### Answer: ")
    answer_text = result[0]["generated_text"].split("### Answer: ")
    answers_nofinetuning.append(answer_text)

100%|███████████████████████████████████████████████████| 6/6 [04:23<00:00, 43.88s/it]


In [21]:
new = []
for i in range(len(answers_nofinetuning)):
    new.append(answers_nofinetuning[i][1])

In [23]:
answers_nofinetuning = new

In [38]:
pd.DataFrame(answers_nofinetuning, columns = ["답변"]).to_csv("ㅎㅎㅎ.csv", encoding = "utf-8-sig")

# RAG with No Finetuning

### rag with train

In [31]:
df = pd.read_csv("datasets/etc/train_final_0213.csv")

In [32]:
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path = "datasets/etc/train_final_0213.csv")
data = loader.load()
data_lst = [i.page_content for i in data]

In [33]:
data_lst

['\ufeff: 0\nid: TRAIN_000\nquestion: 면진장치가 뭐야?\nanswer: 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다.\n: ',
 '\ufeff: 1\nid: TRAIN_000\nquestion: 면진장치가 뭐야?\nanswer: 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. 주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다.\n: ',
 '\ufeff: 2\nid: TRAIN_000\nquestion: 면진장치가 뭐야?\nanswer: 면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다. 이를 통해 건물의 안전성과 안정성을 향상시키고, 지진 등의 외부 충격으로부터 보호하는 역할을 합니다. 지진으로 인한 건물의 피해를 최소화하기 위해 주로 사용됩니다.\n: ',
 '\ufeff: 3\nid: TRAIN_000\nquestion: 면진장치가 뭐야?\nanswer: 면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 최소화해 주는 진동 격리장치입니다. 이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다.\n: ',
 '\ufeff: 4\nid: TRAIN_000\nquestion: 면진장치가 뭐야?\nanswer: 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. 지반으로부터 오는 진동 에너지의 영향을 완화시키기 위해 사용됩니다.\n: ',
 '\ufeff: 5\nid: TRAIN_000\nquestion: 면진장치에 사용되는 주요 기술은 무엇인가요?\nanswer: 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다.\n: ',
 '\ufeff: 6\nid: TRAIN_000\nquestion: 면

### RAG 진행

In [25]:
llm = HuggingFacePipeline(pipeline = pipe)

In [26]:
# Embedding
# distiluse-base-multilingual-cased-v1
modelPath = "jhgan/ko-sbert-nli"
model_kwargs={'device':'cpu'}
encode_kwargs = {'normalize_embeddings':False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [34]:
# Create a FAISS VectorStore
vectorstore = FAISS.from_texts(
    data_lst, embedding=embeddings
)

# Create a Retriever
retriever = vectorstore.as_retriever(
    search_kwargs ={
        "k" : 2
    } 
)

In [39]:
# Chaining a pipeline
chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever,
    return_source_documents=True
)

In [42]:
from tqdm import tqdm
answers_rag = []
answers_doc = []
for text in tqdm(eval):
    chat_history = []
    result = chain({"question": text, "chat_history": chat_history})
    answer_text = result["answer"]
    answer_doc = result["source_documents"]

    answers_rag.append(answer_text)
    answers_doc.append(answer_doc)

 33%|█████████████████                                  | 2/6 [01:05<02:10, 32.58s/it]/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 50%|█████████████████████████▌                         | 3/6 [01:14<01:06, 22.11s/it]/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 67%|██████████████████████████████████                 | 4/6 [01:30<00:39, 19.58s/it]/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 83%|██████████████████████████████████████████▌        | 5/6 [01:43<00:17, 17.16s/it]/usr/local/

In [55]:
pd.DataFrame([answers_rag, answers_doc], index = ["rag답변","참고문서"]).T.to_csv("zzz.csv", encoding = "utf-8-sig")

# 답 뽑아보기

### 임베딩 추출

In [ ]:
from tqdm import tqdm
answers = []
doc = []
for q in tqdm(test["질문"], leave = True):
    prompt = q
    chat_history = []
    result = chain({"question": prompt, "chat_history": chat_history})
    answers.append(result["answer"])
    doc.append(result["source_documents"])

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
vector_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [ ]:
answer_vector = vector_model.encode(answers)

In [ ]:
# 컬럼 형식 맞추
col_list = []
for i in range(512):
    col_list.append(f"vec_{i}")
    
answer_vector = pd.DataFrame(answer_vector, columns = col_list)

In [ ]:
vector_pd = pd.concat([test["id"], answer_vector], axis =1)

In [ ]:
import datetime
suffix = datetime.datetime.now().strftime('%m%d_%H%M')
modelnm = input("해당 모델의 이름을 적으세요,:")
feature = input("해당 모델의 특징을 적으세요.:")
filenm = f"[Vector] {modelnm}_{feature}_{suffix}"

In [ ]:
vector_pd.to_csv(f"datasets/submit/{filenm}.csv", index = False)

### 문서 저장

In [ ]:
test["답변"]=answers
test["참고문서"] = doc

In [ ]:
import datetime
suffix = datetime.datetime.now().strftime('%m%d_%H%M')
filenm = f"[QA] {modelnm}_{feature}_{suffix}"
test.to_csv(f"datasets/submit/{filenm}.csv", index = False, encoding = "utf-8-sig")